Type of Retrieval:
1. Naive Retrieval 
2. Sentench Window Retrieval 
3. Self Query Retrieval
4. Parent Document Retrieval
5. HDE (Hypothetical Document Embedding) 

In [1]:
#Hybrid Search : Naive Retrieval + Keyword Search

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial Intelligence is transforming the world.",
    "Python is a versatile programming language used for various applications.",
    "Traffic sign detection is crucial for autonomous driving systems.",
    "Data science involves analyzing and interpreting complex data."
]

In [5]:
query = "autonomous driving systems"

In [6]:
import re 
def proprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','', text)
    return text